# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	category	title	venue	excerpt	citation	url_slug	paper_url	project_url
2025-02-13	journal	Motion Expressions Guided Video Segmentation Via Effective Motion Information Mining	IEEE Transactions on Emerging Topics in Computational Intelligence	"Motion expressions guided video segmentation is aimed to segment objects in videos according to the given language descriptions about object motion. To accurately segment moving objects across frames, it is important to capture motion information of objects within the entire video. However, the existing method fails to encode object motion information accurately. In this paper, we propose an effective motion information mining framework to improve motion expressions guided video segmentation, named EMIM. It consists of two novel modules, including a hierarchical motion aggregation module and a box-level positional encoding module. Specifically, the hierarchical motion aggregation module is aimed to capture local and global temporal informat

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,category,title,venue,excerpt,citation,url_slug,paper_url,project_url
0,2025-02-13,journal,Motion Expressions Guided Video Segmentation V...,IEEE Transactions on Emerging Topics in Comput...,Motion expressions guided video segmentation i...,"Li Ge, Sun Hanqing, Yang Aiping, Cao Jiale, an...",emim,https://doi.org/10.1109/TETCI.2025.3537936,NaN
1,2024-12-25,journal,Video instance segmentation without using mask...,IEEE Transactions on Multimedia,Video instance segmentation (VIS) is a challen...,"Li Ge, Cao Jiale, Sun Hanqing, Anwer Rao M., X...",mifvis,https://doi.org/10.1109/TMM.2024.3521668,NaN
2,2024-09-30,journal,Transformer-based stereo-aware 3D object detec...,IEEE Transactions on Intelligent Transportatio...,Transformers have shown promising progress in ...,"Sun Hanqing, Pang Yanwei, Cao Jiale, Xie Jin, ...",ts3d,https://doi.org/10.1109/TITS.2024.3462795,NaN
3,2024-04-17,journal,Remote sensing image dehazing via a local cont...,Remote Sensing,Remote sensing image dehazing is a well-known ...,"Nie Jing, Xie Jin, Sun Hanqing. Remote sensing...",lceformer,https://doi.org/10.3390/rs16081422,NaN
4,2023-10-14,journal,Deep intra-image contrastive learning for weak...,Pattern Recognition,Weakly supervised person search aims to perfor...,"Wang Jiabei, Pang Yanwei, Cao Jiale, Sun Hanqi...",dicl,https://doi.org/10.1016/j.patcog.2023.110047,NaN
5,2023-06-20,journal,Supervised biadjacency networks for stereo mat...,Multimedia Tools and Applications,Convolutional neural network (CNN) based stere...,"Sun Hanqing, Han Jungong, Pang Yanwei, Li Xuel...",superbstereo,https://doi.org/10.1007/s11042-023-15362-5,NaN
6,2023-02-05,journal,Semantic-aware self-supervised depth estimatio...,Pattern Recognition Letters,"Besides the 3D object supervision, the auxilia...","Sun Hanqing, Cao Jiale, Pang Yanwei. Semantic-...",s3d,https://doi.org/10.1016/j.patrec.2023.02.006,NaN
7,2020-11-03,journal,TJU-DHD: A diverse high-resolution dataset for...,IEEE Transactions on Image Processing,"Vehicles, pedestrians, and riders are the most...","Pang Yanwei, Cao Jiale, Li Yazhao, Xie Jin, Su...",tju-dhd,https://doi.org/10.1109/TIP.2020.3034487,https://github.com/tjubiit/TJU-DHD
8,2019-07-14,journal,Dual-resolution dual-path convolutional neural...,Sensors,Downsampling input images is a simple trick to...,"Pan Jing, Sun Hanqing, Song Zhanjie, Han Jungo...",dualnet,https://doi.org/10.3390/s19143111,NaN
9,2018-09-03,journal,GlanceNets - efficient convolutional neural ne...,Science China Information Sciences,"Despite the success of CNNs, it is impeded to ...","Sun Hanqing, Pang Yanwei. GlanceNets - efficie...",glancenet,https://doi.org/10.1007/s11432-018-9497-0,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():

    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]

    ## YAML variables

    md = "---\ntitle: \""   + item.title + '"\n'

    md += """collection: publications\n"""

    md += """category: """ + item.category

    md += """\npermalink: /publication/""" + html_filename

    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"

    md += "\ndate: " + str(item.pub_date)

    md += "\nvenue: '" + html_escape(item.venue) + "'"

    if len(str(item.project_url)) > 5:
        md += "\nprojecturl: '" + item.project_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"

    md += "\ncitation: '" + html_escape(item.citation) + "'"

    md += "\n---"

    ## Markdown description for individual page

    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    # if len(str(item.project_url)) > 5:
    #     md += "\n[Project page](" + item.project_url + ")\n"

    # if len(str(item.paper_url)) > 5:
    #     md += "\n[Download paper here](" + item.paper_url + ")\n"

    # md += "\nRecommended citation: " + item.citation

    md_filename = os.path.basename(md_filename)

    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2018-09-03-glancenet.md    2023-10-14-dicl.md
2019-07-14-dualnet.md      2024-04-17-lceformer.md
2020-11-03-tju-dhd.md      2024-09-30-ts3d.md
2023-02-05-s3d.md          2024-12-25-mifvis.md
2023-06-20-superbstereo.md 2025-02-13-emim.md


In [7]:
!cat ../_publications/2024-09-30-ts3d.md

---
title: "Transformer-based stereo-aware 3D object detection from binocular images"
collection: publications
category: journal
permalink: /publication/2024-09-30-ts3d
excerpt: 'Transformers have shown promising progress in various visual object detection tasks, including monocular 2D/3D detection and surround-view 3D detection. More importantly, the attention mechanism in the Transformer model and the 3D information extraction in binocular stereo are both similaritybased. However, directly applying existing Transformer-based detectors to binocular stereo 3D object detection leads to slow convergence and significant precision drops. We argue that a key cause of that defect is that existing Transformers ignore the binocular-stereo-specific image correspondence information. In this paper, we explore the model design of Transformers in binocular 3D object detection, focusing particularly on extracting and encoding task-specific image correspondence information. To achieve this goal, we p